At this point there is a folder that exists with hundreds of thousands of parquet files that I want to start to combine, first into files by day-type and then into files by type.

Similar to in Step 1, files are used identify what work has been done and what work remains to be done. 

In [ ]:
import pandas as pd
import os
import sys
from pyspark.sql.functions import col
from pyspark.sql.functions import StringType
sys.path.insert(1, './.local/lib/python3.9/site-packages')
sys.path.insert(1, '/users/PAS2164/lawr47/.local/bin')

#define the in and out paths and the start and end files.
inLoc='/fs/ess/scratch/PAS2164/James2019Parquet/'
parqStart='/fs/ess/scratch/PAS2164/j19ps.txt'
parqEnd='/fs/ess/scratch/PAS2164/j19pe.txt'
outLoc="/fs/ess/scratch/PAS2164/James2019Parquet2/"

#Define a list of file types
fileTypeList=[]
fileTypeList.append("Alarm")
fileTypeList.append("Wave")
fileTypeList.append("Measurement")

#identify all of the work that there is to do. 
alarmPathList=[]
wavePathList=[]
measPathList=[]
for inFile in os.listdir(inLoc):
    if inFile.split(".")[-1]=="parquet":
        fileType = inFile.split("^")[0]
        #print(inFile,fileType)
        if fileType=="Alarm":
            alarmPathList.append(inLoc+inFile)
        if fileType=="Wave":
            wavePathList.append(inLoc+inFile)
        if fileType=="Measurement":
            measPathList.append(inLoc+inFile)
fullDateList=[]
for file in os.listdir(inLoc):
    fullDateList.append(file.split("^")[3].split(".")[0])
fullDateList=list(set(fullDateList))

startedDateList=[]
writeStart=open(parqStart)
for l in writeStart:
    startedDateList.append(l)
writeStart.close()

#For each Date in the List
for date in fullDateList:
    for ft in fileTypeList:
        newFile=1
        startedDateList=[]
        writeStart=open(parqStart)
        for l in writeStart:
            startedDateList.append(l)
        writeStart.close()
        #Check to see if that date has started
        if (date+","+ft+"\n") in startedDateList:
            newFile=0
        #If it has not started
        if newFile==1:
            print(date,ft)
            #write that you're starting it
            writeStart=open(parqStart,'a')
            writeStart.write((date+","+ft+"\n"))
            writeStart.close()
            dfCombine=0
            if ft=="Alarm":
                localPathList=alarmPathList
            if ft=="Wave":
                localPathList=wavePathList
            if ft=="Measurement":
                localPathList=measPathList
            for fullfilepath in localPathList:
                localDate=(fullfilepath.split("^")[3].split(".")[0])
                if date==localDate:
                    #print(date,fullfilepath)
                    try:
                        tempdf = spark.read.parquet(fullfilepath)
                        if ft=="Alarm":
                            cast_df = tempdf.select(
                            col("patientIdPrimary-id").cast(StringType())
                            ,col("patientIdPrimary-type").cast(StringType())
                            ,col("patientIdPrimary-authority").cast(StringType())
                            ,col("patientNameGiven").cast(StringType())
                            ,col("patientNameFamily").cast(StringType())
                            ,col("assignedLocationCareArea").cast(StringType())
                            ,col("assignedLocationRoom").cast(StringType())
                            ,col("assignedLocationBed").cast(StringType())
                            ,col("polltime").cast(StringType())
                            ,col("alarmName").cast(StringType())
                            ,col("abnormalFlags").cast(StringType())
                            ,col("inactivationState").cast(StringType())
                            ,col("sil").cast(StringType())
                            ,col("setLow").cast(StringType())
                            ,col("setHigh").cast(StringType())
                            ,col("chanValue").cast(StringType()))
                        if ft=="Wave":
                            cast_df = tempdf.select(col("patientIdPrimary-id").cast(StringType())
                            ,col("patientIdPrimary-type").cast(StringType())
                            ,col("patientIdPrimary-authority").cast(StringType())
                            ,col("patientNameGiven").cast(StringType())
                            ,col("patientNameFamily").cast(StringType())
                            ,col("assignedLocationCareArea").cast(StringType())
                            ,col("assignedLocationRoom").cast(StringType())
                            ,col("assignedLocationBed").cast(StringType())
                            ,col("polltime").cast(StringType())
                            ,col("mgname").cast(StringType())
                            ,col("mgGain").cast(StringType())
                            ,col("mgHZ").cast(StringType())
                            ,col("mgwave").cast(StringType())
                            ,col("mguom").cast(StringType())
                            ,col("mgsite").cast(StringType())
                            ,col("mgscale").cast(StringType())
                            ,col("mginvalid").cast(StringType())
                            ,col("mgmissing").cast(StringType())
                            ,col("mgPoints").cast(StringType())
                            ,col("mgPointsBytes").cast(StringType())
                            ,col("mgMin").cast(StringType())
                            ,col("mgMax").cast(StringType())
                            ,col("mgOffset").cast(StringType()))
                        if ft=="Measurement":
                            cast_df = tempdf.select(col("patientIdPrimary-id").cast(StringType())
                            ,col("patientIdPrimary-type").cast(StringType())
                            ,col("patientIdPrimary-authority").cast(StringType())
                            ,col("patientNameGiven").cast(StringType())
                            ,col("patientNameFamily").cast(StringType())
                            ,col("assignedLocationCareArea").cast(StringType())
                            ,col("assignedLocationRoom").cast(StringType())
                            ,col("assignedLocationBed").cast(StringType())
                            ,col("polltime").cast(StringType())
                            ,col("mesname").cast(StringType())
                            ,col("msite").cast(StringType())
                            ,col("muom").cast(StringType())
                            ,col("mtext").cast(StringType()))
                        if dfCombine==0:
                            dfCombine=cast_df
                        if dfCombine!=0:
                            dfCombine=dfCombine.unionByName(cast_df)
                    except:
                        print(fullfilepath,"Corrupt")
            #At this location, dfCombine is the fully combined file and it's ready wo rtie.
            outPath=outLoc+"Ross^"+ft+"^"+str(date)+".parquet"
            try:
                dfCombine.write.option("mergeSchema", "true").parquet(outPath)
                masterFin=open(parqEnd,'a')
                masterFin.write((date+","+ft+"\n"))
                masterFin.close()
            except:
                print(outPath, "Fail")


At this point there is a combined file for every day and for every data type. First use resetrepair to rerun the analysis and fix imcompleted files and then the next step is to build a list of each of those types of files and combine them into three big files.

In [4]:
dailyAlarmPathList=[]
dailyWavePathList=[]
dailyMeasPathList=[]
           
for ft in fileTypeList:
    for file in os.listdir(outLoc):
        if (file.split("^")[1])==ft:
            if ft=="Alarm":
                dailyAlarmPathList.append(outLoc+file)
            if ft=="Wave":
                dailyWavePathList.append(outLoc+file)
            if ft=="Measurement":
                dailyMeasPathList.append(outLoc+file)

In [5]:
dfAlarmDaily = spark.read.parquet(*dailyAlarmPathList)
dfAlarmDaily.write.option("mergeSchema", "true").mode("overwrite").parquet('/fs/ess/scratch/PAS2164/James2019Alarm.parquet')

In [ ]:
dfWaveDaily = spark.read.parquet(*dailyWavePathList)
dfWaveDaily.write.option("mergeSchema", "true").mode("overwrite").parquet('/fs/ess/scratch/PAS2164/James2019Wave.parquet')

In [6]:
dfMeasurementDaily = spark.read.parquet(*dailyMeasPathList)
dfMeasurementDaily.write.option("mergeSchema", "true").mode("overwrite").parquet('/fs/ess/scratch/PAS2164/James2019Measurement.parquet')